# System Resources

| **Item**               | **Grammar**              | **Olaf**                    |
|------------------------|--------------------------|-----------------------------|
| **Number of Nodes**    | 112                      | 100                         |
| **Cores per Node**     | 64                       | 72                          |
| **Threads per Core**   | 1                        | 2 (Hyper-threading enabled) |
| **Memory per Node**    | 512 GB                   | 256 GB                      |
| **Total Cores**        | 7,168                    | 7,200                       |
| **Total Threads**      | 7,168                    | 14,400                      |
| **Total Memory**       | 57,344 GB                | 25,600 GB                   |
 

# Configuration on the Olaf server

| **Item**              | **Configuration**          |
|-----------------------|----------------------------|
| **Supercomputer**     | Olaf                      |
| **Nodes Used**        | 30                        |
| **Total MPI Ranks**   | 600                       |
| **MPI Ranks per Node**| 20                        |
| **CPUs per Task**     | 2 threads (logical cores) |
| **CPUs per Node**     | 40 threads                |
| **Total CPUs Used**   | 1,200 threads             |

# Configuration on the Grammar server to Restart the simulation

The relationship between the number of cores per node, MPI ranks per node, and `cpus-per-task` is defined by the following equation:

$$
\text{Cores per Node} = \text{MPI Ranks per Node} \times \text{cpus-per-task}
$$

When restarting the simulation, **we can NOT change the MPI rank (= 600)**, and `Cores per Node` must not exceed the Grammar server's limit of 64 cores per node. To ensure proper configuration:

$$
64 \geq \text{Cores per Node} = \frac{600}{\text{Nodes Used}} \times \text{cpus-per-task}
$$

In [1]:
class Supercomputer:
    def __init__(self, name, nodes_tot, cores_per_node, threads_per_core, memory_per_node):
        self.name = name
        self.nodes_tot = nodes_tot                # Total nodes in the system
        self.cores_per_node = cores_per_node      # Physical cores per node
        self.threads_per_core = threads_per_core  # Threads per physical core
        self.memory_per_node = memory_per_node    # Memory per node (in GB)

    def calculate_logical_cores_per_node(self):
        """Calculate logical cores per node."""
        return self.cores_per_node * self.threads_per_core

    def calculate_total_logical_cores(self):
        """Calculate total logical cores for the system."""
        return self.calculate_logical_cores_per_node() * self.nodes_tot
    
    def calculte_total_memories(self):
        """Calculate total memories for the system."""
        return self.memory_per_node * self.nodes_tot

    def display_info(self):
        """Display the system's static configuration."""
        logical_cores_per_node = self.calculate_logical_cores_per_node()
        total_logical_cores = self.calculate_total_logical_cores()
        total_memory = self.calculte_total_memories()
        
        print(f"========{self.name}========")
        print(f"Number of Nodes: {self.nodes_tot}")
        print(f"Physical Cores per Node: {self.cores_per_node}")
        print(f"Threads per Core: {self.threads_per_core}")
        print(f"Logical Cores per Node: {logical_cores_per_node}")
        print(f"Total Logical Cores: {total_logical_cores}")
        print(f"Total Memory: {total_memory} GB")
        print()
    
    def calculate_cpus_per_task_limit(self, mpi_ranks_tot, nodes_used):
        """Calculate the upper limit for CPUs per task based on nodes used."""
        logical_cores_per_node = self.calculate_logical_cores_per_node()
        return logical_cores_per_node * nodes_used / mpi_ranks_tot
    

In [2]:
# Create instances for Grammar and Olaf
grammar = Supercomputer(
    name="Grammar",
    nodes_tot=112,
    cores_per_node=64,
    threads_per_core=1,
    memory_per_node=512
)

olaf = Supercomputer(
    name="Olaf",
    nodes_tot=100,
    cores_per_node=72,
    threads_per_core=2,
    memory_per_node=256
)

# for Olaf,
# nodes_tot=210 for the total system
# nodes_tot=100 for the 'large_cpu' particle

In [3]:
# Display system information
grammar.display_info()
olaf.display_info()

========Grammar========
Number of Nodes: 112
Physical Cores per Node: 64
Threads per Core: 1
Logical Cores per Node: 64
Total Logical Cores: 7168
Total Memory: 57344 GB

========Olaf========
Number of Nodes: 100
Physical Cores per Node: 72
Threads per Core: 2
Logical Cores per Node: 144
Total Logical Cores: 14400
Total Memory: 25600 GB



In [4]:
# Calculate CPUs per Task Upper Limit for different nodes_used values
mpi_ranks_tot = 600  # Total MPI ranks
nodes_used_options = [30, 40, 50]  # Example nodes used values

print("====Upper limit of 'cpus-per-task'====")
for nodes_used in nodes_used_options:
    grammar_limit = grammar.calculate_cpus_per_task_limit(mpi_ranks_tot, nodes_used)
    olaf_limit = olaf.calculate_cpus_per_task_limit(mpi_ranks_tot, nodes_used)
    print(f"By using {nodes_used} nodes")
    print(f"  cpus-per-task <= {grammar_limit:.2f} [Grammar]")
    print(f"  cpus-per-task <= {olaf_limit:.2f} [Olaf]")
    print()

====Upper limit of 'cpus-per-task'====
By using 30 nodes
  cpus-per-task <= 3.20 [Grammar]
  cpus-per-task <= 7.20 [Olaf]

By using 40 nodes
  cpus-per-task <= 4.27 [Grammar]
  cpus-per-task <= 9.60 [Olaf]

By using 50 nodes
  cpus-per-task <= 5.33 [Grammar]
  cpus-per-task <= 12.00 [Olaf]

